In [ ]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from joblib import Parallel, delayed

matplotlib.use('Agg')

# Caminho dos diretórios de áudio e espectrogramas
audio_dir = "./data/train_audio/"
spectrogram_dir = "./data/espectogramas"

# Se não existir o diretório ./train_espectogramas, cria
if not os.path.exists(spectrogram_dir):
    os.mkdir(spectrogram_dir)

# Função para criar espectrograma e salvar a imagem
def create_spectrogram(audio_path, output_path):
    # Carrega o arquivo de áudio utilizando a biblioteca librosa
    y, sr = librosa.load(audio_path, sr=None)

    # Calcula a Transformada de Fourier de Curto Prazo (STFT) do sinal de áudio com n_fft=2048, hop_length=1024
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y, n_fft=2048, hop_length=1024)), ref=np.max)
    
    # Cria a figura com o tamanho proporcional
    try:
        # Cria a figura com o tamanho proporcional
        fig = plt.figure(figsize=(D.shape[1]/72, D.shape[0]/72), dpi=72)

        # Plota o espectrograma
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)

        plt.pcolormesh(D)

        # Salva a figura no diretório de espectrogramas
        plt.savefig(output_path, dpi=72, pad_inches=0, bbox_inches='tight')

        # Fecha a figura para liberar recursos de memória
        plt.close()

    except ValueError:
        print("******************************************************")
        print("Erro ao criar espectrograma para o arquivo: " + audio_path)
        print("Número de amostras no arquivo: " + str(len(y)))
        print("******************************************************")

# Função para processar um único arquivo de áudio
def process_audios(audio_dir, espectrogram_dir):
    # percorre e processa os audios do diretório
    for audio_file in os.listdir(audio_dir):
        # Caminho do arquivo de áudio
        audio_path = os.path.join(audio_dir, audio_file)

        # Caminho do arquivo de espectrograma
        output_path = os.path.join(espectrogram_dir, audio_file.replace(".ogg", ".png"))

        # Se o arquivo de espectrograma não existir, cria
        if not os.path.exists(output_path):
            create_spectrogram(audio_path, output_path)
    

# Percorrendo os diretórios de áudio
for key in os.listdir(audio_dir):
    key_dir = os.path.join(audio_dir, key)
    if os.path.isdir(key_dir):
        spectrogram_key_dir = os.path.join(spectrogram_dir, key)
        os.makedirs(spectrogram_key_dir, exist_ok=True)

# Percorrendo os diretórios de áudio
Parallel(n_jobs=-1)(delayed(process_audios)(os.path.join(audio_dir, key), os.path.join(spectrogram_dir, key)) for key in os.listdir(audio_dir))
